In [1]:
from openai import OpenAI
from prompts import colloquial_style_aug_sys_prompt
from functools import cache

import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def colloquial_augment(queries: dict[str, list[str]]):
    res = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": colloquial_style_aug_sys_prompt},
            {"role": "user", "content": json.dumps(queries, indent=4)},
        ],
        response_format={"type": "json_object"},
    )
    return res.choices[0].message.content

In [4]:
with open("queries.jsonl", 'r') as file:
    augmented_queries = []
    for line in file:
        temp = json.loads(line)
        queries = {"customer_queries": temp["customer_queries"]}
        colloquial = colloquial_augment(queries)
        
        colloquial = json.loads(colloquial)
        colloquial.update({"title": temp["title"], "tone": temp["tone"], "style": "colloquial"})
        
        augmented_queries.append(colloquial)

In [5]:
with open("queries.jsonl", 'a') as file:
    for query in augmented_queries:
        json.dump(query, file, ensure_ascii=False)
        file.write('\n')